Let's just generate a loading matrix $C$ and inspect the corresponding SNR bound

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm

from neurofisherSNR.utils import compute_firing_rate
from neurofisherSNR.snr import SNR_bound_instantaneous

d_latent = 2
d_neurons = 320

baseFR = 0.02  # per bin
dt = 0.01  # seconds
b = np.log(0.001 + (baseFR * np.random.rand(d_neurons)))
nT = 1000
x = np.random.randn(nT, d_latent)
CT = np.random.randn(d_latent, d_neurons)
# normalize rows of C to have unit norm
CT = CT / np.linalg.norm(CT, axis=0)
# introduce random gain between 0.5 and 1.5 to each row of C
CT = CT * (0.5 + 0.4 * np.random.rand(d_latent, d_neurons))

SNRdb = SNR_bound_instantaneous(x, CT, b)
print(SNRdb)

In [ ]:
scales = np.logspace(-3, np.log10(4), 30)
snr_list = []
mean_fr_list = []

for scale in scales:
    C_scaled = CT * scale
    snr = SNR_bound_instantaneous(x, C_scaled, b)
    snr_list.append(snr)
    fr = compute_firing_rate(x, C_scaled, b)
    mean_fr = np.mean(fr, axis=0)  # mean per neuron
    mean_fr_list.append(mean_fr)

snr_arr = np.array(snr_list)
mean_fr_arr = np.array(mean_fr_list)  # shape: (n_scales, d_neurons)

fig, ax1 = plt.subplots(figsize=(7, 5))

# Use a colormap to assign different colors to each neuron
colormap = cm.get_cmap('tab20', mean_fr_arr.shape[1])
neuron_colors = [colormap(i) for i in range(d_neurons)]

color1 = 'tab:blue'  # Still use for summary lines
color2 = 'tab:red'

# Plot mean firing rate per neuron (mean and percentiles)
# Plot individual traces for each neuron, with transparency
for i in range(mean_fr_arr.shape[1]):
    ax1.plot(scales, mean_fr_arr[:, i], color=neuron_colors[i], alpha=0.5)

ax1.set_xlabel('C scale factor')
ax1.set_xscale('log')
ax1.set_yscale('log')
ax1.set_ylabel('Mean firing rate per neuron', color=color1)
ax1.tick_params(axis='y', labelcolor=color1)

ax2 = ax1.twinx()
ax2.set_ylabel('SNR (dB)', color=color2)
ax2.plot(scales, snr_arr, color=color2, label='SNR (dB)', marker='o')
ax2.tick_params(axis='y', labelcolor=color2)
ax2.legend(loc='upper left')

plt.title('scaling C with fixed b')
plt.tight_layout()
plt.show()


In [ ]:
# We'll scan a range of bias offsets (additive to b) and plot the effect on SNR and mean firing rate

b_offsets = np.linspace(-2, 2, 30)  # range of bias offsets to scan
snr_b_list = []
mean_fr_b_list = []

for offset in b_offsets:
    b_scan = b + offset  # broadcasted addition
    snr = SNR_bound_instantaneous(x, CT, b_scan)
    snr_b_list.append(snr)
    fr = compute_firing_rate(x, CT, b_scan)
    mean_fr = np.mean(fr, axis=0)  # mean per neuron
    mean_fr_b_list.append(mean_fr)

snr_b_arr = np.array(snr_b_list)
mean_fr_b_arr = np.array(mean_fr_b_list)  # shape: (n_offsets, n_neurons)

fig, ax1 = plt.subplots(figsize=(7, 5))

# Use a colormap to assign different colors to each neuron
colormap = cm.get_cmap('tab20', mean_fr_b_arr.shape[1])
neuron_colors = [colormap(i) for i in range(mean_fr_b_arr.shape[1])]

color1 = 'tab:blue'
color2 = 'tab:red'

# Plot mean firing rate per neuron (mean and percentiles)
for i in range(mean_fr_b_arr.shape[1]):
    ax1.plot(b_offsets, mean_fr_b_arr[:, i], color=neuron_colors[i], alpha=0.5)

ax1.set_xlabel('b offset')
ax1.set_yscale('log')
ax1.set_ylabel('Mean firing rate per neuron', color=color1)
ax1.tick_params(axis='y', labelcolor=color1)

ax2 = ax1.twinx()
ax2.set_ylabel('SNR (dB)', color=color2)
ax2.plot(b_offsets, snr_b_arr, color=color2, label='SNR (dB)', marker='o')
ax2.tick_params(axis='y', labelcolor=color2)
ax2.legend(loc='upper left')

plt.title('scaling b (bias) with fixed C')
plt.tight_layout()
plt.show()

In [ ]:
# plot the firing rate distribution from the log-linear model

fr = compute_firing_rate(x, CT, b)
plt.figure(figsize=(6, 4))
plt.hist(fr.flatten(), bins=50, color='dodgerblue', alpha=0.7, density=True)
plt.xlabel('Firing rate (per bin)')
plt.ylabel('probability density')
plt.title('firing rates distribution')
#plt.yscale('log')
plt.tight_layout()
plt.show()